pip install sentence-transformers transformers scikit-learn pandas matplotlib

In [2]:
import random
from datetime import datetime, timedelta
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from transformers import pipeline

/home/alexander-pool/anaconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
customer_questions = [
    "The site layout is confusing and makes it hard to find what I’m looking for.",
    "I often click on the wrong category because the buttons are too close together.",
    "Navigating between product pages feels cumbersome.",
    "The homepage feels cluttered and overwhelming with too much information.",
    "Dropdown menus are too sensitive and disappear when I try to select something.",
    "There aren’t any breadcrumbs, so I can’t track where I am in the site.",
    "It’s difficult to get back to the homepage from some pages.",
    "The back button doesn’t take me to my previous search results.",
    "It’s hard to find my wishlist or saved items.",
    "There are too many steps involved in getting to the cart from the product page.",
    "The mobile layout feels disorganized compared to the desktop site.",
    "Certain buttons are too small to click on mobile.",
    "Images don’t load properly on the mobile site.",
    "There’s a lag on the mobile version when I scroll.",
    "The mobile site frequently crashes on my phone.",
    "I can't use the mobile site in landscape mode.",
    "Certain features are missing from the mobile version.",
    "Links open in the same tab on mobile, which is annoying.",
    "I have to zoom in on mobile to read the product descriptions.",
    "The mobile app doesn’t let me compare products side by side.",
    "Filters don’t work well and sometimes disappear mid-search.",
    "When I type in a product name, unrelated items show up.",
    "The search bar doesn’t have any auto-suggestions.",
    "I can't filter results by most popular or best-rated items.",
    "The search bar doesn’t let me search for phrases, only single words.",
    "There’s no way to save search criteria for future use.",
    "I wish the search results showed only items that are in stock.",
    "Sometimes my search terms don’t yield any results, even if the product exists.",
    "Search results don’t allow me to view items in grid or list view.",
    "When I spell a word wrong, I don’t get any relevant results.",
    "Product pages are too long and require too much scrolling.",
    "There’s no clear way to see if an item is backordered.",
    "The product descriptions lack technical details.",
    "I’d like to see more photos for each product.",
    "The images are too small and don’t zoom in enough.",
    "There’s no option to read customer Q&A on product pages.",
    "There aren’t any videos demonstrating the products.",
    "Comparing similar items on product pages is difficult.",
    "Product descriptions are missing sizing or dimensions.",
    "I can’t easily find if an item has a warranty on the product page.",
    "The checkout page is overloaded with unnecessary fields.",
    "Guest checkout should be more accessible.",
    "I’m not able to save my payment information securely.",
    "The payment options don’t include enough credit cards.",
    "The promo code field is hard to find in the checkout process.",
    "It’s unclear which currencies are accepted during checkout.",
    "Checkout doesn’t allow splitting payments across different cards.",
    "The order summary doesn’t show tax until the final step.",
    "I wish there was a faster one-click checkout option.",
    "There’s no progress indicator during checkout.",
    "I can’t see estimated delivery times before checkout.",
    "The tracking information updates too slowly.",
    "I’d like to see an eco-friendly shipping option.",
    "Shipping rates aren’t calculated until after I enter my address.",
    "There’s no option to choose a preferred delivery carrier.",
    "Express shipping is too expensive compared to regular shipping.",
    "I can’t pick up the item at a local store location.",
    "It’s unclear if shipping insurance is included.",
    "I want the option to schedule delivery for a specific day.",
    "The tracking number isn’t easy to find on my order confirmation page.",
    "It’s hard to find my past order history in my account.",
    "Updating my address information is too complicated.",
    "I can’t link my account to a social media profile.",
    "The password reset process doesn’t work smoothly.",
    "I don’t see an option to delete my account if I want to.",
    "I can’t update my email preferences from the account page.",
    "The profile picture upload feature isn’t working.",
    "I’d like more privacy options for my account information.",
    "There’s no option to view my account activity log.",
    "It’s difficult to set a default payment method.",
    "I get too many promotional emails after signing up.",
    "There’s no option to receive SMS alerts for order updates.",
    "I don’t get notifications when an out-of-stock item is available again.",
    "Email receipts take too long to arrive.",
    "Push notifications on the app don’t work consistently.",
    "I’d like to receive alerts when prices drop on wishlist items.",
    "It’s hard to customize my notification settings.",
    "I don’t get notified about special members-only sales.",
    "My order status emails don’t always include tracking info.",
    "I receive duplicate notifications for the same event.",
    "Customer service live chat isn’t available during weekends.",
    "The help center articles are too vague and unhelpful.",
    "It takes too long to reach a human representative.",
    "There’s no option for a callback instead of waiting on hold.",
    "The chatbot doesn’t understand my questions.",
    "Email responses from support take too long.",
    "The customer service number isn’t toll-free.",
    "There’s no online forum for customers to discuss issues.",
    "I’d like to see an option for video call support.",
    "Support reps don’t have detailed knowledge of products.",
    "Most reviews are outdated and don’t reflect current product quality.",
    "I can’t filter reviews by star rating.",
    "There’s no option to see verified customer reviews.",
    "It’s difficult to write a review on the mobile app.",
    "I wish I could ‘like’ or ‘dislike’ reviews that are helpful.",
    "There’s no way to see photos posted by other reviewers.",
    "I want to be able to sort reviews by most recent.",
    "Reviews don’t give enough detail about sizing or fit.",
    "It’s not easy to flag inappropriate reviews.",
    "The review system doesn’t allow me to add video reviews.",
    "I can’t move items between my wishlist and cart easily.",
    "There’s no option to share my wishlist with friends.",
    "The cart doesn’t save items if I leave the site and come back.",
    "I can’t see stock availability directly in my wishlist.",
    "There’s no notification if an item in my cart is about to sell out.",
    "I wish the wishlist had subcategories to organize my items.",
    "Items in my cart disappear after a short time.",
    "I want to save my cart for later and return to it easily.",
    "There’s no option to apply a promo code to wishlist items.",
    "The wishlist doesn’t allow for price tracking.",
    "I’d like to see a dark mode option for the site.",
    "There’s no option to personalize the homepage based on my interests.",
    "The site doesn’t support multi-language options.",
    "I wish I could donate part of my purchase to charity at checkout.",
    "It’s unclear how to unsubscribe from promotional emails.",
    "The terms and conditions are hard to find and understand.",
    "There’s no FAQ section for common questions.",
    "I’d like to see sustainability information for each product.",
    "I want the ability to refer friends and earn rewards.",
    "There’s no loyalty program for frequent buyers."
]



In [8]:
# Convert to DataFrame
df = pd.DataFrame(customer_questions, columns=["question"])


# 2. Embedding the Questions using SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # Lightweight embedding model
questions = df['question'].tolist()
embeddings = model.encode(questions)

# 3. Clustering the Questions using KMeans
num_clusters = 10 # Adjust number of clusters as needed
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df['cluster'] = kmeans.fit_predict(embeddings)

# Print sample clusters
print("Sample Questions by Cluster:")
for i in range(num_clusters):
    print(f"\nCluster {i}:")
    print(df[df['cluster'] == i]['question'].sample(3).tolist())

Sample Questions by Cluster:

Cluster 0:
['The checkout page is overloaded with unnecessary fields.', 'It’s difficult to set a default payment method.', 'It’s unclear which currencies are accepted during checkout.']

Cluster 1:
['Navigating between product pages feels cumbersome.', 'The mobile app doesn’t let me compare products side by side.', 'The images are too small and don’t zoom in enough.']

Cluster 2:
['The wishlist doesn’t allow for price tracking.', 'I wish the search results showed only items that are in stock.', 'I can’t see stock availability directly in my wishlist.']

Cluster 3:
['I wish I could ‘like’ or ‘dislike’ reviews that are helpful.', 'I can’t filter reviews by star rating.', 'Most reviews are outdated and don’t reflect current product quality.']

Cluster 4:
['The search bar doesn’t have any auto-suggestions.', 'The back button doesn’t take me to my previous search results.', 'The search bar doesn’t let me search for phrases, only single words.']

Cluster 5:
['Th

In [16]:
# Summarizing the most common questions per cluster
# 5. Text Summarization for Each Cluster Using LLM
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
cluster_summaries = {}
for i in range(num_clusters):
    cluster_questions = df[df['cluster'] == i]['question'].tolist()
    cluster_text = " ".join(cluster_questions[:5])  # Concatenate the top 5 questions for summarization
    summary = summarizer(cluster_text, max_length=35, min_length=30, do_sample=False)
    cluster_summaries[i] = summary[0]['summary_text']
    print(f"Cluster {i} Summary: {cluster_summaries[i]}")

# 6. Augmenting LLM Suggestions with Clustering and Time-Series Data
# Mock LLM model for generating suggestions
suggestion_generator = pipeline("text-generation", model="meta-llama/Llama-3.2-1B-Instruct")

# Function to augment suggestions with cluster summaries and time-series insights
def suggest_enhancements(query, cluster_summaries):
    # Generate a prompt for the LLM including cluster summaries and time trends
    prompt = (
        f"Based on the following customer feedback themes: {cluster_summaries}\n"
        "Please suggest improvements for the eCommerce website to address these issues."
    )

    # Generate suggestions using the LLM
    response = suggestion_generator(prompt, max_length=1000)[0]['generated_text']
    return response

# Example Usage
sample_query = "checkout issues"
augmented_suggestion = suggest_enhancements(sample_query, cluster_summaries)
print("LLM Suggestion:", augmented_suggestion)


Cluster 0 Summary: The checkout page is overloaded with unnecessary fields. It’s unclear which currencies are accepted during checkout. Checkout doesn’t allow splitting payments across different
Cluster 1 Summary: Navigating between product pages feels cumbersome. I have to zoom in on mobile to read the product descriptions. The mobile app doesn't let me compare products side by
Cluster 2 Summary: I wish the search results showed only items that are in stock. I’d like to receive alerts when prices drop on wishlist items. There’
Cluster 3 Summary: Most reviews are outdated and don’t reflect current product quality. It’s difficult to write a review on the mobile app. I wish I could
Cluster 4 Summary: The search bar doesn’t let me search for phrases, only single words. The back button doesn't take me to my previous search results. The search
Cluster 5 Summary: There aren't any breadcrumbs, so I can't track where I am in the site. There's no clear way to see if an item is back
Cluster 6 Su

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


LLM Suggestion: Based on the following customer feedback themes: {0: 'The checkout page is overloaded with unnecessary fields. It’s unclear which currencies are accepted during checkout. Checkout doesn’t allow splitting payments across different', 1: "Navigating between product pages feels cumbersome. I have to zoom in on mobile to read the product descriptions. The mobile app doesn't let me compare products side by", 2: 'I wish the search results showed only items that are in stock. I’d like to receive alerts when prices drop on wishlist items. There’', 3: 'Most reviews are outdated and don’t reflect current product quality. It’s difficult to write a review on the mobile app. I wish I could', 4: "The search bar doesn’t let me search for phrases, only single words. The back button doesn't take me to my previous search results. The search", 5: "There aren't any breadcrumbs, so I can't track where I am in the site. There's no clear way to see if an item is back", 6: "There aren't any vid